# Regresja: modele liniowe, KNN oraz wielomianowe

W tym laboratorium generujemy sztuczne dane do regresji, a następnie porównujemy różne podejścia do modelowania zależności między zmiennymi:
- regresję liniową,
- regresję k-najbliższych sąsiadów (KNN),
- regresję wielomianową (od 2 do 5 stopnia).

Celem jest analiza dopasowania modeli oraz porównanie błędów średniokwadratowych (MSE) dla zbioru treningowego i testowego.

Na końcu zapisujemy wszystkie modele oraz wyniki MSE do plików `.pkl`, które mogą być potem wykorzystane do dalszej analizy lub wizualizacji.

📘 Co to jest regresja?
Regresja to jedno z podstawowych zagadnień w uczeniu maszynowym i statystyce. Celem regresji jest modelowanie zależności między zmienną objaśnianą (y) a jedną lub więcej zmiennymi objaśniającymi (X), aby na podstawie nowych danych wejściowych przewidywać wartość wyjściową.

W odróżnieniu od klasyfikacji (gdzie przewidujemy etykietę/kategorię), regresja przewiduje wartość liczbową.

📈 Przykład zastosowania regresji:
przewidywanie cen mieszkań na podstawie powierzchni, lokalizacji, liczby pokoi,

prognozowanie temperatury na podstawie danych historycznych,

modelowanie zależności między ilością reklamy a sprzedażą.



## Generowanie danych i wizualizacja

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

size = 300
X = np.random.rand(size) * 5 - 2.5
w4, w3, w2, w1, w0 = 1, 2, 1, -4, 2
y = w4*(X**4) + w3*(X**3) + w2*(X**2) + w1*X + w0 + np.random.randn(size)*8 - 4

df = pd.DataFrame({'x': X, 'y': y})
df.to_csv("dane_do_regresji.csv", index=None)
df.plot.scatter(x='x', y='y')
plt.title("Wygenerowane dane regresyjne")
plt.show()


## Przygotowanie struktury na wyniki MSE i podział danych

In [ ]:

from sklearn.model_selection import train_test_split

mse = pd.DataFrame(columns=["train_mse", "test_mse"],
                   index=["lin_reg", "knn_3_reg", "knn_5_reg",
                          "poly_2_reg", "poly_3_reg", "poly_4_reg", "poly_5_reg"])

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)


## Regresja liniowa

In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_train)
plt.scatter(X_train, y_train, color='blue', label='Dane')
plt.plot(X_train, y_pred, color='red', label='Regresja liniowa')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Regresja liniowa')
plt.legend()
plt.show()

mse_train = mean_squared_error(y_train, y_pred)
y_test_pred = lin_reg.predict(X_test)
mse_test = mean_squared_error(y_test, y_test_pred)
mse.loc["lin_reg"] = [mse_train, mse_test]


## Regresja KNN (k=3)

In [ ]:

from sklearn.neighbors import KNeighborsRegressor

knn_reg3 = KNeighborsRegressor(n_neighbors=3)
knn_reg3.fit(X_train, y_train)
y_train_pred_knn3 = knn_reg3.predict(X_train)
y_test_pred_knn3 = knn_reg3.predict(X_test)

plt.clf()
plt.scatter(X_train, y_train, c='blue')
X_new = np.arange(-3, 3, 0.001).reshape(-1, 1)
plt.plot(X_new, knn_reg3.predict(X_new), c="red")
plt.title("KNN Regressor (k=3)")
plt.show()

mse.loc["knn_3_reg"] = [mean_squared_error(y_train, y_train_pred_knn3),
                        mean_squared_error(y_test, y_test_pred_knn3)]


## Regresja KNN (k=5)

In [ ]:

knn_reg5 = KNeighborsRegressor(n_neighbors=5)
knn_reg5.fit(X_train, y_train)
y_train_pred_knn5 = knn_reg5.predict(X_train)
y_test_pred_knn5 = knn_reg5.predict(X_test)

plt.clf()
plt.scatter(X_train, y_train, c='blue')
plt.plot(X_new, knn_reg5.predict(X_new), c="red")
plt.title("KNN Regressor (k=5)")
plt.show()

mse.loc["knn_5_reg"] = [mean_squared_error(y_train, y_train_pred_knn5),
                        mean_squared_error(y_test, y_test_pred_knn5)]


## Regresja wielomianowa (stopień 2–5)

In [ ]:

from sklearn.preprocessing import PolynomialFeatures

models = []

for degree in range(2, 6):
    poly_features = PolynomialFeatures(degree=degree, include_bias=False)
    X_poly_train = poly_features.fit_transform(X_train)
    model = LinearRegression()
    model.fit(X_poly_train, y_train)

    y_train_pred = model.predict(X_poly_train)
    X_poly_test = poly_features.transform(X_test)
    y_test_pred = model.predict(X_poly_test)

    mse.loc[f"poly_{degree}_reg"] = [
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)
    ]

    # Wykres
    plt.clf()
    plt.scatter(X_train, y_train, c='blue', label="Dane treningowe")
    X_new = np.arange(-3, 3, 0.001).reshape(-1, 1)
    poly_X_new = poly_features.transform(X_new)
    y_new_pred_poly = model.predict(poly_X_new)
    plt.plot(X_new, y_new_pred_poly, c="r", label=f"Regresja wielomianowa {degree} stopnia")
    plt.legend()
    plt.title(f"Regresja wielomianowa stopnia {degree}")
    plt.show()

    models.append((model, poly_features))


## Zapis wyników

In [ ]:

import pickle

with open("mse.pkl", "wb") as file:
    pickle.dump(mse, file)

reg_models = [(lin_reg, None), (knn_reg3, None), (knn_reg5, None)] + models

with open("reg.pkl", "wb") as file:
    pickle.dump(reg_models, file)
